AgriWatch AI: Technical Documentation
Edge-Optimized Plant Pathology Classification



1. Project Overview

AgriWatch AI is a compact computer vision framework engineered for plant disease classification. The system targets 16 distinct health states across Pepper, Potato, and Tomato species using the PlantVillage dataset.

The primary objective was the development of a high-efficiency pipeline capable of delivering local inference on mobile and embedded hardware, and edge devices such as Raspberry Bi. The project evaluates the efficacy of Transfer Learning, Progressive Fine-Tuning, and Model Quantization in reducing computational overhead while maintaining high predictive accuracy.

    Backend Architecture: MobileNetV3

    Dataset: PlantVillage (Kaggle)

    Target Precision: TFLite Float16

2. Experimental Workflows

Two complementary experimental workflows were conducted to evaluate optimization strategies:

    Workflow 1: Quantization-Aware Training (QAT): Implemented to explore the impact of simulating low-precision hardware constraints during the training process.

    Workflow 2: Static Quantization (FP16): A multi-stage conversion pipeline (ONNX → TensorFlow → TFLite) designed to produce a stable, deployment-ready binary.

3. Workflow 1: Quantization-Aware Training (QAT)
3.1 Baseline Performance

Evaluation of a pretrained MobileNetV3 directly on the PlantVillage dataset yielded a validation accuracy of 9.63% (Weighted F1: 0.09). This established that the stock backbone required significant domain-specific fine-tuning to differentiate between specific plant pathologies.
3.2 Stage 1: Frozen Classifier Head

The initial adaptation phase involved freezing the MobileNetV3 feature extractor and training only the classifier head (3 epochs, learning rate 1e−3). This stage achieved a validation accuracy of 92.29% (Weighted F1: 0.92), demonstrating that transfer learning successfully leveraged the backbone's existing visual features.
3.3 Stage 2 through Stage 7: Progressive Unfreezing & QAT

To refine the model for quantized environments, the system underwent a progressive unfreezing schedule combined with layer fusion (Conv2d + BatchNorm + ReLU):

    Stage 2: Initial unfreezing achieved 65.87% accuracy.

    Stage 4: Mid-stage training reached 77.17% accuracy.

    Stage 7: Peak fine-tuning reached 87.12% accuracy, with a final test set result of 77.70%.

Observation on QAT Reliability: While QAT provided a theoretical framework for 8-bit optimization, the workflow was ultimately deprecated due to the complexity and fragility of the exportation pipeline. The transition from PyTorch’s quantized operators through the ONNX-to-TensorFlow bridge introduced significant parity risks and non-deterministic behavior during conversion. Furthermore, the specialized nature of the QAT graph made future model refinement and iterative updates appears unreliable for a production environment. Consequently, the static conversion path was prioritized to ensure a maintainable and stable deployment lifecycle.

The Stage 1 model (92.29% accuracy) was selected as the source for the production pipeline. The model was exported via the following chain to ensure cross-platform compatibility: PyTorch ⮕ ONNX ⮕ TensorFlow ⮕ TFLite (Float16)

The resulting model size is 3.1 MB, specifically optimized for ARM-based edge CPUs.
4.2 Performance Analysis

The TFLite Float16 model maintained a validation accuracy of 92.10% and a Weighted F1-score of 0.92.

    High-Performing Classes: Potato_healthy, Tomato_YellowLeaf_Curl_Virus, and Pepper_healthy (F1 scores between 0.96 and 1.0).

    Performance Bottlenecks: Tomato_Early_blight recorded a moderate F1 score of 0.77, indicating visual similarity with other class variants.

Conclusion: Float16 quantization effectively preserved the model’s predictive performance while meeting the storage constraints required for edge deployment.
5. Deployment Implementation

AgriWatch AI is designed for immediate execution on hardware such as the Raspberry Pi. The implementation utilizes the tflite-runtime for minimized dependency overhead.

Inference Workflow:

    Preprocessing: Resizing and normalization of input imagery to match MobileNetV3 requirements.

    Execution: Invocation of the TFLite Interpreter.

    Classification: Mapping of the output tensor to the 16 target labels.

System Requirements: pip install tflite-runtime pillow numpy
6. Key Observations

    Transfer Learning Efficacy: Training only the classifier head provided a strong performance baseline.

    Progressive Fine-tuning: Gradual unfreezing of feature layers improved convergence during the QAT experiments.

    Quantization Strategy: While QAT offers a complex look at weight robustness, Static TFLite quantization (Float16) provided the most reliable path to a 3.1 MB deployable model with minimal accuracy degradation.

7. Conclusion

AgriWatch AI demonstrates a complete engineering pipeline for creating a high-accuracy, edge-optimized plant disease classifier. By balancing transfer learning and precision quantization, the project resulted in a 3.1 MB model achieving 92.10% accuracy, fully operational for practical agricultural monitoring on embedded devices.